In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("bank.csv")
df['deposit'] = df['deposit'].map({'no': 0, 'yes': 1})

## One Hot Encode and Prepare the data

In [3]:
X = df.drop('deposit', axis=1)  # Drop the target variable to get the features
y = df['deposit']  # Select only the target variable

# Display the original DataFrame
print("Original DataFrame:")
print(df.head())

# Perform one-hot encoding on the features
X = pd.get_dummies(X, dtype=int)

# Display the encoded features DataFrame
print("\nFeatures DataFrame after one-hot encoding:")
print(X.head())

Original DataFrame:
   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome  deposit  
0    5   may      1042         1     -1         0  unknown        1  
1    5   may      1467         1     -1         0  unknown        1  
2    5   may      1389         1     -1         0  unknown        1  
3    5   may       579         1     -1         0  unknown        1  
4    5   may       673         2     -1         0  unknown        1  

Features DataFrame after one-hot encoding:
   age  balance  day  duratio

In [4]:
for column in X:
    X[column] = (X[column] - X[column].mean()) / X[column].std()

In [5]:
print(X)
y["deposit"] = (y["deposit"] - y["deposit"].mean()) / y["deposit"].std()
print(y)

            age   balance       day  duration  campaign     pdays  previous  \
0      1.491438  0.252514 -1.265689  1.930139 -0.554144 -0.481162 -0.363244   
1      1.239620 -0.459953 -1.265689  3.154470 -0.554144 -0.481162 -0.363244   
2     -0.019470 -0.080157 -1.265689  2.929770 -0.554144 -0.481162 -0.363244   
3      1.155681  0.293749 -1.265689  0.596339 -0.554144 -0.481162 -0.363244   
4      1.071742 -0.416858 -1.265689  0.867132 -0.186777 -0.481162 -0.363244   
...         ...       ...       ...       ...       ...       ...       ...   
11157 -0.690984 -0.473595  0.515627 -0.331272 -0.554144 -0.481162 -0.363244   
11158 -0.187348 -0.246647  0.040610 -0.832527  0.547956 -0.481162 -0.363244   
11159 -0.774923 -0.464914  0.396873 -0.622230 -0.186777 -0.481162 -0.363244   
11160  0.148409 -0.473905 -0.909426 -1.045705 -0.186777  1.109521  1.818250   
11161 -0.607045 -0.473905 -0.790671  0.737497 -0.554144 -0.481162 -0.363244   

       job_admin.  job_blue-collar  job_entrepreneu

KeyError: 'deposit'